In [1]:
import transformers
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu

In [2]:
# Load CoNaLa dataset
#dataset = load_dataset("grail_qa", "conala")  # Using Hugging Face Datasets for convenience
dataset = load_dataset("neulab/conala")

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question_id', 'intent', 'rewritten_intent', 'snippet'],
        num_rows: 2379
    })
    test: Dataset({
        features: ['question_id', 'intent', 'rewritten_intent', 'snippet'],
        num_rows: 500
    })
})


In [4]:
import torch

In [5]:
# Create pipeline for Code Llama
model = transformers.pipeline(
    "text-generation",
    model="codellama/CodeLlama-7b-Instruct-hf",
    torch_dtype=torch.float32,
    device_map="auto"  # Use GPU if available
)

2024-03-18 11:14:54.716872: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 11:14:54.716928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 11:14:54.720586: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Iterate through testing set
bleu_scores = []
for example in dataset["test"]:
    
    prompt = example["intent"]  

    # Reference code for BLEU score calculation
    reference_code = example["snippet"]  # Expected code snippet

    # Generate code using the model pipeline
    output = model(prompt, max_length=50)
    #output = model(prompt, max_length=20, batch_size=1, truncation=True)
    generated_text = output[0].get("generated_text")  # Access generated text
    
    '''
    if generated_text is None:
        print("Generated text not found in model output:", output)
        continue

    # Ensure generated_text is a string
    if not isinstance(generated_text, str):
        print("Generated text is not a string:", generated_text)
        continue
    '''

    # Calculate BLEU score
    bleu = sentence_bleu([reference_code.split()], generated_text.split())
    bleu_scores.append(bleu)

In [7]:
# Print average BLEU score
print(f"Average BLEU score: {sum(bleu_scores) / len(bleu_scores)}")

Average BLEU score: 0.0952474061356062
